# データサイエンス100本ノック（構造化データ加工編） - Python

## はじめに
- 初めに以下のセルを実行してください
- 必要なライブラリのインポートとデータベース（PostgreSQL）からのデータ読み込みを行います
- pandas等、利用が想定されるライブラリは以下セルでインポートしています
- その他利用したいライブラリがあれば適宜インストールしてください（"!pip install ライブラリ名"でインストールも可能）
- 処理は複数回に分けても構いません
- 名前、住所等はダミーデータであり、実在するものではありません

In [75]:
import pandas as pd
import numpy as np
import math

df_customer = pd.read_csv('data/customer.csv')
df_category = pd.read_csv('data/category.csv')
df_product = pd.read_csv('data/product.csv')
df_receipt = pd.read_csv('data/receipt.csv')
df_store = pd.read_csv('data/store.csv')
df_geocode = pd.read_csv('data/geocode.csv')
# df_geocode = pd.read_csv('data/geocode.csv', sep='\t', converters={'town':str})

/Users/kento/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# 演習問題

---
> ★P-003: レシート明細データ（df_receipt）から売上年月日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示せよ。ただし、sales_ymdをsales_dateに項目名を変更して抽出すること。

In [19]:
df_p_003 = df_receipt[['sales_ymd', 'customer_id', 'product_cd','amount']] #複数行を抽出する場合はdf[[]]となる
df_p_003 = df_p_003.rename(columns={'sales_ymd':'sales_date'})
print(df_p_003.head(10))

   sales_date     customer_id  product_cd  amount
0    20181103  CS006214000001  P070305012     158
1    20181118  CS008415000097  P070701017      81
2    20170712  CS028414000014  P060101005     170
3    20190205  ZZ000000000000  P050301001      25
4    20180821  CS025415000050  P060102007      90
5    20190605  CS003515000195  P050102002     138
6    20181205  CS024514000042  P080101005      30
7    20190922  CS040415000178  P070501004     128
8    20170504  ZZ000000000000  P071302010     770
9    20191010  CS027514000015  P071101003     680


---
>★ P-007: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上2,000以下

In [20]:
df_p_007 = df_receipt[['sales_ymd', 'customer_id', 'product_cd','amount']] 
df_p_007 = df_p_007.rename(columns={'sales_ymd':'sales_date'})
df_p_007 = df_p_007[(df_p_007['customer_id'] == "CS018205000001") & (df_p_007['amount'] > 1000) & (df_p_007['amount'] < 2000)]
print(df_p_007)

       sales_date     customer_id  product_cd  amount
72254    20180911  CS018205000001  P071401005    1100


---
>★ P-013: 顧客データ（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まるデータを全項目抽出し、10件表示せよ。

In [21]:
df_p_013 = df_customer[(df_customer['status_cd'].str.startswith(('A','B','C','D','E','F')))] #startswithで複数の場合はタプル()で指定する
print(df_p_013.head(10))

       customer_id customer_name  gender_cd gender   birth_day  age postal_cd  \
2   CS031415000172       宇多田 貴美子          1     女性  1976-10-04   42  151-0053   
6   CS015414000103         奥野 陽子          1     女性  1977-08-09   41  136-0073   
12  CS011215000048         芦田 沙耶          1     女性  1992-02-01   27  223-0062   
15  CS029415000023         梅田 里穂          1     女性  1976-01-17   43  279-0043   
21  CS035415000029         寺沢 真希          9     不明  1977-09-27   41  158-0096   
32  CS031415000106        宇野 由美子          1     女性  1970-02-26   49  151-0053   
33  CS029215000025         石倉 美帆          1     女性  1993-09-28   25  279-0022   
40  CS033605000005         猪股 雄太          0     男性  1955-12-05   63  246-0031   
44  CS033415000229        板垣 菜々美          1     女性  1977-11-07   41  246-0021   
53  CS008415000145         黒谷 麻緒          1     女性  1977-06-27   41  157-0067   

                     address application_store_cd  application_date  \
2        東京都渋谷区代々木**********         

---
> ★P-016: 店舗データ（df_store）から、電話番号（tel_no）が3桁-3桁-4桁のデータを全項目表示せよ。


In [22]:
pattern = "\d{3}-\d{3}-\d{4}" #正規表現を用いて電話番号を指定する（\d：数字, {n}：n回繰り返し）
df_p_016 = df_store[df_store['tel_no'].str.contains(pattern)]
print(df_p_016)

   store_cd store_name  prefecture_cd prefecture              address  \
0    S12014       千草台店             12        千葉県      千葉県千葉市稲毛区千草台一丁目   
1    S13002       国分寺店             13        東京都         東京都国分寺市本多二丁目   
2    S14010        菊名店             14       神奈川県      神奈川県横浜市港北区菊名一丁目   
3    S14033       阿久和店             14       神奈川県    神奈川県横浜市瀬谷区阿久和西一丁目   
4    S14036     相模原中央店             14       神奈川県        神奈川県相模原市中央二丁目   
7    S14040       長津田店             14       神奈川県  神奈川県横浜市緑区長津田みなみ台五丁目   
9    S14050      阿久和西店             14       神奈川県    神奈川県横浜市瀬谷区阿久和西一丁目   
11   S13052        森野店             13        東京都          東京都町田市森野三丁目   
12   S14028       二ツ橋店             14       神奈川県       神奈川県横浜市瀬谷区二ツ橋町   
16   S14012      本牧和田店             14       神奈川県        神奈川県横浜市中区本牧和田   
18   S14046       北山田店             14       神奈川県     神奈川県横浜市都筑区北山田一丁目   
19   S14022        逗子店             14       神奈川県         神奈川県逗子市逗子一丁目   
20   S14011      日吉本町店             14       神奈川県   

---
>★ P-022: レシート明細データ（df_receipt）の顧客ID（customer_id）に対し、ユニーク件数をカウントせよ。

In [23]:
df_p_022 = df_receipt['customer_id'].nunique()
print(df_p_022)


8307


---
>★ P-026: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上年月日（sales_ymd）と古い売上年月日を求め、両者が異なるデータを10件表示せよ。

In [54]:
df_p_026 = df_receipt.groupby('customer_id').agg({'sales_ymd':['max','min']}).reset_index()

# カラム名を変更
df_p_026.columns = ['customer_id', 'sales_ymd_max', 'sales_ymd_min']

# 売上年月日が異なるデータを10件表示
df_p_026.query('sales_ymd_max != sales_ymd_min').head(10)



# 過去回答（不正解）
# # 顧客ID（customer_id）ごとに最も新しい売上年月日（sales_ymd）と古い売上年月日を求める
# df_p_026_group = pd.DataFrame()
# df_p_026_group['recent_ymd'] = df_receipt.groupby('customer_id')['sales_ymd'].max()
# df_p_026_group['old_ymd'] = df_receipt.groupby('customer_id')['sales_ymd'].min()

# # 売上年月日が異なるデータの'customer_id''recent_ymd''old_ymd'でDataFrameを新規作成する
# df_p_026_extract = pd.DataFrame()
# df_p_026_extract = df_p_026_group[~df_p_026_group['recent_ymd'].isin(df_p_026_group['old_ymd'])]
# df_p_026_extract['customer_id'] = df_p_026_extract.index #customer_idがindexとなっているので、列として追加し直す
# df_p_026_extract.reset_index(inplace=True, drop=True) #indexを振り直しておく

# #新規作成したDataFrameをもとに、本来のDataFrameからデータを抽出する
# df_p_026_extract_list = df_p_026_extract['customer_id']
# df_p_026 = df_receipt[df_receipt['customer_id'].isin(df_p_026_extract_list)]
# print(df_p_026.head(10))


,customer_id,sales_ymd_max,sales_ymd_min
1,CS001114000005,20190731,20180503
2,CS001115000010,20190405,20171228
3,CS001205000004,20190625,20170914
4,CS001205000006,20190224,20180207
13,CS001214000009,20190902,20170306
14,CS001214000017,20191006,20180828
16,CS001214000048,20190929,20171109
17,CS001214000052,20190617,20180208
20,CS001215000005,20181021,20170206
21,CS001215000040,20171022,20170214


---
>★ P-027: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、降順でTOP5を表示せよ。

In [25]:
# 店舗コード（store_cd）ごとに売上金額（amount）の平均を計算する
df_p_027_group = pd.DataFrame()
df_p_027_group['amount_avg'] = df_receipt.groupby('store_cd')['amount'].mean()
df_p_027_group['store_cd'] = df_p_027_group.index #store_cdがindexとなっているので、列として追加し直す
df_p_027_group.reset_index(inplace=True, drop=True) #indexを振り直しておく

#降順でTOP5を表示する
df_p_027 = df_p_027_group.sort_values(by='amount_avg', ascending=False).head(5)
print(df_p_027)


    amount_avg store_cd
28  402.867470   S13052
12  351.111960   S13015
7   350.915519   S13003
30  348.791262   S14010
5   348.470386   S13001


---
> ★P-036: レシート明細データ（df_receipt）と店舗データ（df_store）を内部結合し、レシート明細データの全項目と店舗データの店舗名（store_name）を10件表示せよ。

In [26]:
#レシート明細データ（df_receipt）と店舗データ（df_store）を店舗コード（store_cd）で内部結合する
df_p_036_merge = pd.DataFrame()
df_p_036_merge = df_receipt.merge(df_store, on='store_cd', how='inner')
# print(df_p_036_merge.head(10))

#レシート明細データの全項目と店舗データの店舗名（store_name）に絞り、10件のみ表示される
df_p_036_columns = list(df_receipt.columns.values) + ["store_name"]
df_p_036 = df_p_036_merge[df_p_036_columns]
print(df_p_036.head(10))


   sales_ymd  sales_epoch store_cd  receipt_no  receipt_sub_no  \
0   20181103   1541203200   S14006         112               1   
1   20181116   1542326400   S14006         112               2   
2   20170118   1484697600   S14006        1162               1   
3   20190524   1558656000   S14006        1192               1   
4   20190419   1555632000   S14006         112               2   
5   20181119   1542585600   S14006        1152               2   
6   20171211   1512950400   S14006        1132               2   
7   20191021   1571616000   S14006        1112               2   
8   20170710   1499644800   S14006        1132               2   
9   20190805   1564963200   S14006         112               1   

      customer_id  product_cd  quantity  amount store_name  
0  CS006214000001  P070305012         1     158       葛が谷店  
1  ZZ000000000000  P080401001         1      48       葛が谷店  
2  CS006815000006  P050406035         1     220       葛が谷店  
3  CS006514000034  P060104003

---
>★ P-045: 顧客データ（df_customer）の生年月日（birth_day）は日付型でデータを保有している。これをYYYYMMDD形式の文字列に変換し、顧客ID（customer_id）とともに10件表示せよ。

In [27]:
df_p_045 = pd.DataFrame()
df_p_045 = df_customer

#生年月日（birth_day）を日付型からYYYYMMDD形式の文字列に変換
df_p_045['birth_day'] = pd.to_datetime(df_customer['birth_day']).dt.strftime("%Y%m%d")
print(df_p_045[['customer_id','birth_day']].head(10))

      customer_id birth_day
0  CS021313000114  19810429
1  CS037613000071  19520401
2  CS031415000172  19761004
3  CS028811000001  19330327
4  CS001215000145  19950329
5  CS020401000016  19740915
6  CS015414000103  19770809
7  CS029403000008  19730817
8  CS015804000004  19310502
9  CS033513000180  19620711


---
> ★P-051: レシート明細データ（df_receipt）の売上エポック秒を日付型に変換し、「日」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。なお、「日」は0埋め2桁で取り出すこと。

In [62]:
df_p_051 = pd.DataFrame()

# エポック秒になっているので、unit='s'を指定する
df_p_051_time = pd.to_datetime(df_receipt['sales_epoch'], unit='s').rename('sales_day')
df_p_051 = pd.concat([df_receipt[['receipt_no', 'receipt_sub_no']],df_p_051_time.dt.strftime('%d')], axis=1)

print(df_p_051.head(10))

# 過去回答（不正解）
# df_p_051 = pd.DataFrame()
# df_p_051 = df_receipt

# #レシート明細データ（df_receipt）の売上エポック秒を日付型に変換する
# df_p_051['sales_epoch'] = pd.to_datetime(df_receipt['sales_epoch']).dt.strftime("%Y%m%d")

# #「日」だけ取り出し、0埋め2桁で取り出すこと
# df_p_051['sales_epoch_day'] = pd.to_datetime(df_receipt['sales_epoch']).dt.strftime("%d")

# #レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。「日」は0埋め2桁で表示する。
# print(df_p_051[['receipt_no','receipt_sub_no','sales_epoch_day']].head(10))


   receipt_no  receipt_sub_no sales_day
0         112               1        03
1        1132               2        18
2        1102               1        12
3        1132               1        05
4        1102               2        21
5        1112               1        05
6        1102               2        05
7        1102               1        22
8        1112               2        04
9        1102               1        10


---
>★ P-052: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計の上、売上金額合計に対して2,000円以下を0、2,000円より大きい金額を1に二値化し、顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [64]:
df_p_052 = pd.DataFrame()
df_p_052_group = pd.DataFrame()

#レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計する
df_p_052_group['amount_sum'] = df_receipt.groupby('customer_id')['amount'].sum()
df_p_052_group['customer_id'] = df_p_052_group.index #store_cdがindexとなっているので、列として追加し直す
df_p_052_group.reset_index(inplace=True, drop=True) #indexを振り直しておく

#売上金額合計で二値化する：2,000円以下を0/2,000円より大きい金額を1
#loc関数：行と列のラベルを指定し、その要素を取得または変更する
df_p_052_group.loc[(df_p_052_group['amount_sum'] > 2000), 'amount_flag'] = 1
df_p_052_group.loc[(df_p_052_group['amount_sum'] <= 2000), 'amount_flag'] = 0
df_p_052_group['amount_flag'] = df_p_052_group['amount_flag'].astype(int)

#顧客IDが"Z"から始まるのものは非会員を表すため、除外処理を行う
#顧客ID、売上金額合計とともに10件表示する
df_p_052 = df_p_052_group.query('not customer_id.str.startswith("Z")', engine='python')
df_p_052 = df_p_052.reindex(columns=['customer_id','amount_sum','amount_flag'])
print(df_p_052.head(10))


      customer_id  amount_sum  amount_flag
0  CS001113000004        1298            0
1  CS001114000005         626            0
2  CS001115000010        3044            1
3  CS001205000004        1988            0
4  CS001205000006        3337            1
5  CS001211000025         456            0
6  CS001212000027         448            0
7  CS001212000031         296            0
8  CS001212000046         228            0
9  CS001212000070         456            0


---
>★ P-053: 顧客データ（df_customer）の郵便番号（postal_cd）に対し、東京（先頭3桁が100〜209のもの）を1、それ以外のものを0に二値化せよ。さらにレシート明細データ（df_receipt）と結合し、全期間において売上実績のある顧客数を、作成した二値ごとにカウントせよ。

In [65]:
df_p_053 = pd.DataFrame()
df_p_053_df_customer = pd.DataFrame()
df_p_053_merge = pd.DataFrame()

# 顧客データ（df_customer）の郵便番号（postal_cd）から先頭3桁を抜き出し、int型に変換
df_p_053_df_customer = df_customer
df_p_053_df_customer['postal_cd_prefecture'] = df_p_053_df_customer['postal_cd'].str[:3]
df_p_053_df_customer['postal_cd_prefecture'] = df_p_053_df_customer['postal_cd_prefecture'].astype(int)

# 東京（先頭3桁が100〜209のもの）を1、それ以外を0に二値化する
df_p_053_df_customer.loc[((df_p_053_df_customer['postal_cd_prefecture'] >= 100) & (df_p_053_df_customer['postal_cd_prefecture'] <= 209)), 'postal_cd_flag'] = 1
df_p_053_df_customer.loc[(df_p_053_df_customer['postal_cd_flag'] != 1), 'postal_cd_flag'] = 0
df_p_053_df_customer['postal_cd_flag'] = df_p_053_df_customer['postal_cd_flag'].astype(int)

# レシート明細データ（df_receipt）と結合する
df_p_053_merge = df_receipt.merge(df_p_053_df_customer, on='customer_id', how='inner')

# 全期間において売上実績のある顧客数を、作成した二値ごとにカウントする
print(df_p_053_merge.groupby('postal_cd_flag')['customer_id'].nunique())


postal_cd_flag
0    3906
1    4400
Name: customer_id, dtype: int64


---
>★ P-054: 顧客データ（df_customer）の住所（address）は、埼玉県、千葉県、東京都、神奈川県のいずれかとなっている。都道府県毎にコード値を作成し、顧客ID、住所とともに10件表示せよ。値は埼玉県を11、千葉県を12、東京都を13、神奈川県を14とすること。

In [31]:
df_p_054 = pd.DataFrame()
df_p_054_df_customer = pd.DataFrame()

# 顧客データ（df_customer）の住所（address）をもとに都道府県コードを割り当てる / 埼玉県:11、千葉県:12、東京都:13、神奈川県:14
df_p_054_df_customer = df_customer
df_p_054_df_customer.loc[((df_p_054_df_customer['address'].str.contains('埼玉県'))),'prefecture_cd'] = 11
df_p_054_df_customer.loc[((df_p_054_df_customer['address'].str.contains('千葉県'))),'prefecture_cd'] = 12
df_p_054_df_customer.loc[((df_p_054_df_customer['address'].str.contains('東京都'))),'prefecture_cd'] = 13
df_p_054_df_customer.loc[((df_p_054_df_customer['address'].str.contains('神奈川県'))),'prefecture_cd'] = 14
df_p_054_df_customer['prefecture_cd'] = df_p_054_df_customer['prefecture_cd'].astype(int)

df_p_054 = df_p_054_df_customer

# 顧客ID、住所とともに10件表示する
print(df_p_054[['customer_id','address','prefecture_cd']].head(10))

      customer_id                 address  prefecture_cd
0  CS021313000114    神奈川県伊勢原市粟窪**********             14
1  CS037613000071      東京都江東区南砂**********             13
2  CS031415000172     東京都渋谷区代々木**********             13
3  CS028811000001  神奈川県横浜市泉区和泉町**********             14
4  CS001215000145     東京都大田区仲六郷**********             13
5  CS020401000016      東京都板橋区若木**********             13
6  CS015414000103      東京都江東区北砂**********             13
7  CS029403000008      千葉県浦安市海楽**********             12
8  CS015804000004      東京都江東区北砂**********             13
9  CS033513000180  神奈川県横浜市旭区善部町**********             14


---
>★ P-055: レシート明細（df_receipt）データの売上金額（amount）を顧客ID（customer_id）ごとに合計し、その合計金額の四分位点を求めよ。その上で、顧客ごとの売上金額合計に対して以下の基準でカテゴリ値を作成し、顧客ID、売上金額合計とともに10件表示せよ。カテゴリ値は順に1〜4とする。
>
> - 最小値以上第1四分位未満 ・・・ 1を付与
> - 第1四分位以上第2四分位未満 ・・・ 2を付与
> - 第2四分位以上第3四分位未満 ・・・ 3を付与
> - 第3四分位以上 ・・・ 4を付与

In [32]:
df_p_055 = pd.DataFrame()
df_p_055_group = pd.DataFrame()

# レシート明細（df_receipt）データの売上金額（amount）を顧客ID（customer_id）ごとに合計する
df_p_055_group['amount_sum'] = df_receipt.groupby('customer_id')['amount'].sum()
df_p_055_group['customer_id'] = df_p_055_group.index #store_cdがindexとなっているので、列として追加し直す
df_p_055_group.reset_index(inplace=True, drop=True) #indexを振り直しておく
# print(df_p_055_group.head(20))

# 合計金額の四分位点を求める
df_p_055_quantile = df_p_055_group['amount_sum'].quantile([0, 0.25, 0.5, 0.75, 1.0])
p_055_quantile_min = df_p_055_quantile.loc[0.00]
p_055_quantile_1st = df_p_055_quantile.loc[0.25]
p_055_quantile_2nd = df_p_055_quantile.loc[0.50]
p_055_quantile_3rd = df_p_055_quantile.loc[0.75]
# print(df_p_055_quantile)
# print(p_055_quantile_min)

# 顧客ごとの売上金額合計に対してカテゴリ値を作成する / 最小値以上第1四分位未満:1、第1四分位以上第2四分位未満:2、第2四分位以上第3四分位未満:3、第3四分位以上:4
df_p_055_group.loc[((df_p_055_group['amount_sum'] >= p_055_quantile_min) & (df_p_055_group['amount_sum'] < p_055_quantile_1st)),'category'] = 1
df_p_055_group.loc[((df_p_055_group['amount_sum'] >= p_055_quantile_1st) & (df_p_055_group['amount_sum'] < p_055_quantile_2nd)),'category'] = 2
df_p_055_group.loc[((df_p_055_group['amount_sum'] >= p_055_quantile_2nd) & (df_p_055_group['amount_sum'] < p_055_quantile_3rd)),'category'] = 3
df_p_055_group.loc[((df_p_055_group['amount_sum'] >= p_055_quantile_3rd)),'category'] = 4
df_p_055_group['category'] = df_p_055_group['category'].astype(int)

# カラムの整列
df_p_055 = df_p_055_group.reindex(columns=['customer_id','amount_sum','category'])

# 顧客IDと売上金額合計とカテゴリ値を10件表示する
print(df_p_055.head(10))

      customer_id  amount_sum  category
0  CS001113000004        1298         2
1  CS001114000005         626         2
2  CS001115000010        3044         3
3  CS001205000004        1988         3
4  CS001205000006        3337         3
5  CS001211000025         456         1
6  CS001212000027         448         1
7  CS001212000031         296         1
8  CS001212000046         228         1
9  CS001212000070         456         1


---
> ★P-056: 顧客データ（df_customer）の年齢（age）をもとに10歳刻みで年代を算出し、顧客ID（customer_id）、生年月日（birth_day）とともに10件表示せよ。ただし、60歳以上は全て60歳代とすること。年代を表すカテゴリ名は任意とする。

In [66]:
df_p_056 = pd.DataFrame()
df_p_056 = df_customer

# 顧客データ（df_customer）の年齢（age）をもとに10歳刻みで年代を算出する
# 60歳以上は全て60歳代とし、年代を表すカテゴリ名は任意
df_p_056.loc[((df_p_056['age'] >= 0) & (df_p_056['age'] < 10)),'age_category'] = '0s'
df_p_056.loc[((df_p_056['age'] >= 10) & (df_p_056['age'] < 20)),'age_category'] = '10s'
df_p_056.loc[((df_p_056['age'] >= 20) & (df_p_056['age'] < 30)),'age_category'] = '20s'
df_p_056.loc[((df_p_056['age'] >= 30) & (df_p_056['age'] < 40)),'age_category'] = '30s'
df_p_056.loc[((df_p_056['age'] >= 40) & (df_p_056['age'] < 50)),'age_category'] = '40s'
df_p_056.loc[((df_p_056['age'] >= 50) & (df_p_056['age'] < 60)),'age_category'] = '50s'
df_p_056.loc[((df_p_056['age'] >= 60)),'age_category'] = 'over60s'

# 顧客ID（customer_id）、生年月日（birth_day）とともに10件表示せよ。
print(df_p_056[['customer_id','birth_day','age_category']].head(10))


      customer_id   birth_day age_category
0  CS021313000114  1981-04-29          30s
1  CS037613000071  1952-04-01      over60s
2  CS031415000172  1976-10-04          40s
3  CS028811000001  1933-03-27      over60s
4  CS001215000145  1995-03-29          20s
5  CS020401000016  1974-09-15          40s
6  CS015414000103  1977-08-09          40s
7  CS029403000008  1973-08-17          40s
8  CS015804000004  1931-05-02      over60s
9  CS033513000180  1962-07-11          50s


---
>★ P-057: 056の抽出結果と性別コード（gender_cd）により、新たに性別×年代の組み合わせを表すカテゴリデータを作成し、10件表示せよ。組み合わせを表すカテゴリの値は任意とする。

In [34]:
#056の抽出結果を代入する
df_p_057 = pd.DataFrame()
df_p_057 = df_p_056

# 性別コード（gender_cd）をもとに性別のタグを作成
df_p_057.loc[((df_p_057['gender_cd'] == 0)),'gender_tag'] = 'M'
df_p_057.loc[((df_p_057['gender_cd'] == 1)),'gender_tag'] = 'F'
df_p_057.loc[((df_p_057['gender_cd'] == 9)),'gender_tag'] = 'U'

# 新たに性別×年代の組み合わせを表すカテゴリデータを作成：性別のタグと年代を結合する
df_p_057['sex_age_segment'] = df_p_057['gender_tag'].str.cat(df_p_057['age_category'], sep='_')

# 結果を10件表示する
print(df_p_057[['customer_id', 'birth_day', 'sex_age_segment']].head(10))

      customer_id birth_day sex_age_segment
0  CS021313000114  19810429           F_30s
1  CS037613000071  19520401       U_over60s
2  CS031415000172  19761004           F_40s
3  CS028811000001  19330327       F_over60s
4  CS001215000145  19950329           F_20s
5  CS020401000016  19740915           M_40s
6  CS015414000103  19770809           F_40s
7  CS029403000008  19730817           M_40s
8  CS015804000004  19310502       M_over60s
9  CS033513000180  19620711           F_50s


---
>★P-058: 顧客データ（df_customer）の性別コード（gender_cd）をダミー変数化し、顧客ID（customer_id）とともに10件表示せよ。


In [35]:
df_p_058 = pd.DataFrame()
df_p_058_dummies = pd.DataFrame()

# 顧客データ（df_customer）の性別コード（gender_cd）をダミー変数化する
df_p_058 = df_customer
df_p_058.isnull().all() #ダミー変数化する前に欠損値がないかどうか確認
df_p_058_dummies = pd.get_dummies(df_p_058['gender_cd'])
df_p_058_dummies = df_p_058_dummies.rename(columns={0:'gender_cd_M',1:'gender_cd_F',9:'gender_cd_U'})
df_p_058_dummies['customer_id'] = df_customer['customer_id']

# カラムの整列
df_p_058 = df_p_058_dummies.reindex(columns=['customer_id','gender_cd_M','gender_cd_F','gender_cd_U'])

# 顧客ID（customer_id）とともに10件表示する
print(df_p_058.head(10))

      customer_id  gender_cd_M  gender_cd_F  gender_cd_U
0  CS021313000114            0            1            0
1  CS037613000071            0            0            1
2  CS031415000172            0            1            0
3  CS028811000001            0            1            0
4  CS001215000145            0            1            0
5  CS020401000016            1            0            0
6  CS015414000103            0            1            0
7  CS029403000008            1            0            0
8  CS015804000004            1            0            0
9  CS033513000180            0            1            0


---
> ★P-059: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を平均0、標準偏差1に標準化して顧客ID、売上金額合計とともに10件表示せよ。標準化に使用する標準偏差は、分散の平方根、もしくは不偏分散の平方根のどちらでも良いものとする。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [36]:
df_p_059 = pd.DataFrame()
df_p_059_group = pd.DataFrame()

# 必要なライブラリのインポート｜平均0・分散1に正規化（標準化）するSciPyのscipy.stats.zscore()関数を用いる
import scipy.stats

# レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計
df_p_059_group['amount_sum'] = df_receipt.groupby('customer_id')['amount'].sum()
df_p_059_group['customer_id'] = df_p_059_group.index #customer_idがindexとなっているので、列として追加し直す
df_p_059_group.reset_index(inplace=True, drop=True) #indexを振り直しておく
df_p_059_group = df_p_059_group.reindex(columns=['customer_id','amount_sum']) # カラムの整列

# 売上金額合計を平均0、標準偏差1に標準化
# 顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算
df_p_059 =df_p_059_group.query('not customer_id.str.startswith("Z")', engine='python')
df_p_059['amount_sum_ standardized'] = scipy.stats.zscore(df_p_059['amount_sum'])

# 顧客ID、売上金額合計とともに10件表示する
print(df_p_059.head(10))

      customer_id  amount_sum  amount_sum_ standardized
0  CS001113000004        1298                 -0.459378
1  CS001114000005         626                 -0.706390
2  CS001115000010        3044                  0.182413
3  CS001205000004        1988                 -0.205749
4  CS001205000006        3337                  0.290114
5  CS001211000025         456                 -0.768879
6  CS001212000027         448                 -0.771819
7  CS001212000031         296                 -0.827691
8  CS001212000046         228                 -0.852686
9  CS001212000070         456                 -0.768879


/var/folders/ll/63945nf12rl06b030t73hsmw0000gn/T/ipykernel_63863/2046033036.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_p_059['amount_sum_ standardized'] = scipy.stats.zscore(df_p_059['amount_sum'])


---
> ★P-063: 商品データ（df_product）の単価（unit_price）と原価（unit_cost）から各商品の利益額を算出し、結果を10件表示せよ。


In [37]:
df_p_063 = pd.DataFrame()
df_p_063 = df_product

# 商品データ（df_product）の単価（unit_price）と原価（unit_cost）から各商品の利益額を算出する｜単価（unit_price）- 原価（unit_cost）
df_p_063['unite_profit'] = df_p_063['unit_price'] - df_p_063['unit_cost']

# 結果を10件表示する
print(df_p_063.head(10))

   product_cd  category_major_cd  category_medium_cd  category_small_cd  \
0  P040101001                  4                 401              40101   
1  P040101002                  4                 401              40101   
2  P040101003                  4                 401              40101   
3  P040101004                  4                 401              40101   
4  P040101005                  4                 401              40101   
5  P040101006                  4                 401              40101   
6  P040101007                  4                 401              40101   
7  P040101008                  4                 401              40101   
8  P040101009                  4                 401              40101   
9  P040101010                  4                 401              40101   

   unit_price  unit_cost  unite_profit  
0       198.0      149.0          49.0  
1       218.0      164.0          54.0  
2       230.0      173.0          57.0  
3       24

---
> ★P-065: 商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。ただし、1円未満は切り捨てること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [74]:
df_p_065 = pd.DataFrame()
df_p_065 = df_product

# 商品データ（df_product）の各商品について、利益率が30%となる新たな単価を算出
# 1円未満は切り捨てる
p65_profit_rate = 0.3
p65_cost_rate = 1.0 - p65_profit_rate # 利益率から原価率を定義
df_p_065.dropna(subset=['unit_cost']) # 原価が欠損している場合には、利益率が算出できないため、原価（unit_cost）の列で欠損値処理を実行
df_p_065['new_price'] = df_p_065['unit_cost'] / p65_cost_rate # 原価（unit_cost）から新しい単価を算出
df_p_065['new_price'] = df_p_065['new_price'].round()

# 利益率を算出する
df_p_065['profit_rate'] = (df_p_065['new_price'] - df_p_065['unit_cost'])/df_p_065['unit_price']  #利益率を算出する

# 結果を10件表示し、利益率がおよそ30％付近であることを確認する
print(df_p_065[['product_cd','unit_price','unit_cost','new_price','profit_rate']].head(10))

   product_cd  unit_price  unit_cost  new_price  profit_rate
0  P040101001       198.0      149.0      213.0     0.323232
1  P040101002       218.0      164.0      234.0     0.321101
2  P040101003       230.0      173.0      247.0     0.321739
3  P040101004       248.0      186.0      266.0     0.322581
4  P040101005       268.0      201.0      287.0     0.320896
5  P040101006       298.0      224.0      320.0     0.322148
6  P040101007       338.0      254.0      363.0     0.322485
7  P040101008       420.0      315.0      450.0     0.321429
8  P040101009       498.0      374.0      534.0     0.321285
9  P040101010       580.0      435.0      621.0     0.320690


---
> ★P-069: レシート明細データ（df_receipt）と商品データ（df_product）を結合し、顧客毎に全商品の売上金額合計と、カテゴリ大区分コード（category_major_cd）が"07"（瓶詰缶詰）の売上金額合計を計算の上、両者の比率を求めよ。抽出対象はカテゴリ大区分コード"07"（瓶詰缶詰）の売上実績がある顧客のみとし、結果を10件表示せよ。


In [39]:
df_p_069 = pd.DataFrame()
df_p_069_merge = pd.DataFrame()
df_p_069_group = pd.DataFrame()

# レシート明細データ（df_receipt）と商品データ（df_product）を結合する
df_p_069_merge = df_receipt.merge(df_product, on='product_cd', how='inner')

# 顧客毎に全商品の売上金額合計を算出する
df_p_069_group['amount_sum'] = df_p_069_merge.groupby('customer_id')['amount'].sum()

# カテゴリ大区分コード（category_major_cd）が"07"（瓶詰缶詰）の売上金額合計を算出する
# groupbyは直後に.mean() .max() .sum()がくることが想定されているので、先に.loc[]で取り出す
df_p_069_group['amount_category_major_7_sum'] = df_p_069_merge.loc[df_p_069_merge['category_major_cd'] == 7,['customer_id','amount']].groupby('customer_id').sum('amount')
df_p_069_group.fillna({'amount_category_major_7_sum': 0})

# データテーブルを成形する
df_p_069_group['customer_id'] = df_p_069_group.index #customer_idがindexとなっているので、列として追加し直す
df_p_069_group.reset_index(inplace=True, drop=True) #indexを振り直しておく
df_p_069_group = df_p_069_group.reindex(columns=['customer_id','amount_sum','amount_category_major_7_sum']) # カラムの整列
df_p_069_group = df_p_069_group.dropna(subset=['amount_category_major_7_sum']) #カテゴリ大区分コード"07"（瓶詰缶詰）の売上実績がない顧客を削除する

# 全商品の売上合計と瓶詰缶詰の売上合計比率を算出する
df_p_069_group['rate_category_major_7'] = df_p_069_group['amount_category_major_7_sum'] / df_p_069_group['amount_sum']
df_p_069 = df_p_069_group

# 抽出対象はカテゴリ大区分コード"07"（瓶詰缶詰）の売上実績がある顧客のみとし、結果を10件表示せよ。
print(df_p_069.head(10))

       customer_id  amount_sum  amount_category_major_7_sum  \
0   CS001113000004        1298                       1298.0   
1   CS001114000005         626                        486.0   
2   CS001115000010        3044                       2694.0   
3   CS001205000004        1988                        346.0   
4   CS001205000006        3337                       2004.0   
6   CS001212000027         448                        200.0   
7   CS001212000031         296                        296.0   
8   CS001212000046         228                        108.0   
9   CS001212000070         456                        308.0   
11  CS001213000018         243                        145.0   

    rate_category_major_7  
0                1.000000  
1                0.776358  
2                0.885020  
3                0.174044  
4                0.600539  
6                0.446429  
7                1.000000  
8                0.473684  
9                0.675439  
11               0.596708 

---
> ★P-070: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過日数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。


In [40]:
df_p_070 = pd.DataFrame()
df_p_070_merge = pd.DataFrame()

# レシート明細データ（df_receipt）と顧客データ（df_customer）を結合｜顧客ID（customer_id）をキーにする
df_p_070_merge = df_receipt.merge(df_customer, on='customer_id', how='inner')

# 計算が可能なように日付のデータ型を揃える｜sales_ymd:int64、application_date:int64
df_p_070_merge.dtypes
df_p_070_merge['sales_ymd_new']= pd.to_datetime(df_p_070_merge['sales_ymd'].astype(str))
df_p_070_merge['application_date_new'] = pd.to_datetime(df_p_070_merge['application_date'].astype(str))

# 会員申込日（application_date）からの経過日数を計算する
df_p_070_merge['elapsed_days'] = df_p_070_merge['sales_ymd_new'] - df_p_070_merge['application_date_new']
df_p_070 = df_p_070_merge.drop(columns=['sales_ymd','application_date'])
df_p_070 = df_p_070.rename(columns={'sales_ymd_new': 'sales_ymd', 'application_date_new': 'application_date'})

# 顧客ID（customer_id）、売上日、会員申込日とともに10件表示
print(df_p_070[['customer_id','sales_ymd','application_date','elapsed_days']].head(10))

      customer_id  sales_ymd application_date elapsed_days
0  CS006214000001 2018-11-03       2015-02-01    1371 days
1  CS006214000001 2017-05-09       2015-02-01     828 days
2  CS006214000001 2017-06-08       2015-02-01     858 days
3  CS006214000001 2017-06-08       2015-02-01     858 days
4  CS006214000001 2018-10-28       2015-02-01    1365 days
5  CS006214000001 2018-10-28       2015-02-01    1365 days
6  CS006214000001 2017-05-09       2015-02-01     828 days
7  CS006214000001 2019-09-08       2015-02-01    1680 days
8  CS006214000001 2018-01-31       2015-02-01    1095 days
9  CS006214000001 2017-07-05       2015-02-01     885 days


---
> ★P-075: 顧客データ（df_customer）からランダムに1%のデータを抽出し、先頭から10件表示せよ。

In [41]:
df_p_075 = pd.DataFrame()
df_p_075 = df_customer.sample(frac=0.01)
print(df_p_075.head(10))

          customer_id customer_name  gender_cd gender birth_day  age  \
16062  CS019514000087        平塚 美智子          1     女性  19670606   51   
12796  CS018512000116          立石 茜          1     女性  19581217   60   
9808   CS001715000084         伴 だん吉          9     不明  19420912   76   
2269   CS030415000034         玉置 南朋          1     女性  19690128   50   
70     CS031403000002         栗田 誠治          0     男性  19700817   48   
17522  CS003312000225        篠崎 沙知絵          1     女性  19831012   35   
17917  CS004314000230         野際 愛子          1     女性  19850918   33   
2354   CS040714000036          緒方 涼          1     女性  19440507   74   
6251   CS004613000040        長田 ひろみ          1     女性  19521124   66   
18422  CS022413000051        山内 貴美子          1     女性  19700208   49   

      postal_cd                  address application_store_cd  \
16062  176-0004      東京都練馬区小竹町**********               S13019   
12796  204-0011      東京都清瀬市下清戸**********               S13018   
9808   146-0

---
> ★P-076: 顧客データ（df_customer）から性別コード（gender_cd）の割合に基づきランダムに10%のデータを層化抽出し、性別コードごとに件数を集計せよ。


In [42]:
df_p_076 = pd.DataFrame()
df_p_076 = df_customer

#顧客データ（df_customer）から性別コード（gender_cd）の割合に基づきランダムに10%のデータを層化抽出
df_p_076 = df_p_076.groupby('gender_cd', group_keys=False).apply(lambda x: x.sample(frac=0.1))

# 性別コードごとに件数を集計せよ。
print(df_p_076['gender_cd'].value_counts())

1    1792
0     298
9     107
Name: gender_cd, dtype: int64


---
> ★P-077: レシート明細データ（df_receipt）の売上金額を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。なお、外れ値は売上金額合計を対数化したうえで平均と標準偏差を計算し、その平均から3σを超えて離れたものとする（自然対数と常用対数のどちらでも可）。結果は10件表示せよ。

In [71]:
df_p_077 = pd.DataFrame()
df_p_077_group = pd.DataFrame()

#必要なライブラリーをインポートする
import scipy.stats

# レシート明細データ（df_receipt）の売上金額を顧客単位に合計
df_p_077_group = df_receipt.query('not customer_id.str.startswith("Z")', engine='python').groupby('customer_id').agg({'amount':'sum'}).reset_index()

# 売上金額合計を対数化する
df_p_077_group['amount_sum_standardized'] = scipy.stats.zscore(df_p_077_group['amount'])

# 平均と標準偏差を計算する
df_p_077_avg = df_p_077_group['amount_sum_standardized'].mean()
df_p_077_std = df_p_077_group['amount_sum_standardized'].std()
print(df_p_077_avg)
print(df_p_077_std)

# 平均から3σを超えて離れたものを外れ値とする
# 結果は10件表示
df_p_077 = df_p_077_group.query('amount_sum_standardized > (@df_p_077_avg + 3)')
print(df_p_077.head(10))


2.598317526569759e-16
1.0000602028837735
         customer_id  amount  amount_sum_standardized
332   CS001605000009   18925                 6.019921
1755  CS006415000147   12723                 3.740202
1817  CS006515000023   18372                 5.816651
1833  CS006515000125   12575                 3.685800
1841  CS006515000209   11373                 3.243972
1870  CS007115000006   11528                 3.300946
1941  CS007514000056   13293                 3.949721
1943  CS007514000094   15735                 4.847347
1951  CS007515000107   11188                 3.175970
1997  CS007615000026   11959                 3.459372


---
> ★P-078: レシート明細データ（df_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を第1四分位と第3四分位の差であるIQRを用いて、「第1四分位数-1.5×IQR」を下回るもの、または「第3四分位数+1.5×IQR」を超えるものとする。結果は10件表示せよ。

In [44]:
df_p_078 = pd.DataFrame()
df_p_078_group = pd.DataFrame()

# レシート明細データ（df_receipt）の売上金額（amount）を顧客単位に合計する
# 顧客IDが"Z"から始まるのものは非会員を表すため、除外する
df_p_078_group = df_receipt.query('not customer_id.str.startswith("Z")', engine='python').groupby('customer_id').agg({'amount':'sum'}).reset_index()

# 合計金額の四分位点を求める
# IQR（第1四分位と第3四分位の差）を求める
# 外れ値の基準を設ける｜「第1四分位数-1.5×IQR」と「第3四分位数+1.5×IQR」
df_p_078_quantile = df_p_078_group['amount'].quantile([0, 0.25, 0.5, 0.75, 1.0])
p_078_quantile_1st = df_p_078_quantile.loc[0.25]
p_078_quantile_3rd = df_p_078_quantile.loc[0.75]
p_078_iqr = p_078_quantile_3rd - p_078_quantile_1st

p_078_std_1st = p_078_quantile_1st - 1.5*p_078_iqr
p_078_std_3rd = p_078_quantile_3rd + 1.5*p_078_iqr

# 合計した売上金額の外れ値を抽出する
# 結果は10件表示
df_p_078 = df_p_078_group.query('amount < @p_078_std_1st | amount > @p_078_std_3rd')
print(df_p_078.head(10))



         customer_id  amount
98    CS001414000048    8584
332   CS001605000009   18925
549   CS002415000594    9568
1180  CS004414000181    9584
1558  CS005415000137    8734
1733  CS006414000001    9156
1736  CS006414000029    9179
1752  CS006415000105   10042
1755  CS006415000147   12723
1757  CS006415000157   10648


---
> ★P-081: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの平均値で補完した新たな商品データを作成せよ。なお、平均値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。


In [76]:
df_p_081 = pd.DataFrame()
df_p_081_row = pd.DataFrame()

# 単価（unit_price）と原価（unit_cost）の各平均値を算出する
# 1円未満を丸める（四捨五入または偶数への丸め）
df_p_081_row = df_product
p_081_unit_price_avg = round(df_p_081_row['unit_price'].mean(), -1)
p_081_unit_cost_avg = round(df_p_081_row['unit_cost'].mean(),-1)

# 単価（unit_price）と原価（unit_cost）の欠損値を各平均値で補完し処理する
# 新たな商品データを作成する
df_p_081 = df_p_081_row.fillna({'unit_price':p_081_unit_price_avg, 'unit_cost':p_081_unit_cost_avg})

#各項目について欠損が生じていないことも確認する
print(df_p_081.isnull().any())


product_cd            False
category_major_cd     False
category_medium_cd    False
category_small_cd     False
unit_price            False
unit_cost             False
dtype: bool


---
> ★P-083: 単価（unit_price）と原価（unit_cost）の欠損値について、各商品のカテゴリ小区分コード（category_small_cd）ごとに算出した中央値で補完した新たな商品データを作成せよ。なお、中央値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

In [77]:
#写経での回答
df_p_083 = pd.DataFrame()

# 各商品のカテゴリ小区分コード（category_small_cd）ごとに中央値を算出する
# 中央値を1円未満を丸める（四捨五入または偶数への丸め）
df_p_083_mid = round(df_product.groupby('category_small_cd').agg({'unit_price':'median', 'unit_cost':'median'}),-1).reset_index()
df_p_083_mid.columns = ['category_small_cd', 'median_price', 'median_cost']


# 元のデータと中央値のDataFrameを内部結合する
df_p_083 = pd.merge(df_product, df_p_083_mid, how='inner', on='category_small_cd')


# 単価（unit_price）と原価（unit_cost）の欠損値を各中央値で補完し処理する
# unit_price(下記コードではx[0]がunit_price)に欠損値があった場合は、median_price(下記コードではx[1]がmedian_price)を適用し、そうでない場合は。unit_priceそのまま
# axis=1にすることで行方向への処理にさせる。
df_p_083['unit_price'] = df_p_083[['unit_price', 'median_price']].apply(lambda x: x[1] if np.isnan(x[0]) else x[0], axis=1)
df_p_083['unit_cost'] = df_p_083[['unit_cost', 'median_cost']].apply(lambda x: x[1] if np.isnan(x[0]) else x[0], axis=1)

# 各項目について欠損が生じていないことも確認する
p_083_selected_columns = df_p_083.columns[~df_p_083.columns.isin(['median_price', 'median_cost'])]
# print(df_p_083[p_083_selected_columns])
print(df_p_083.isnull().sum())

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            0
unit_cost             0
median_price          0
median_cost           0
dtype: int64


---
> ★P-084: 顧客データ（df_customer）の全顧客に対して全期間の売上金額に占める2019年売上金額の割合を計算し、新たなデータを作成せよ。ただし、売上実績がない場合は0として扱うこと。そして計算した割合が0超のものを抽出し、結果を10件表示せよ。また、作成したデータに欠損が存在しないことを確認せよ。

In [47]:
df_p_084 = pd.DataFrame()
df_p_084_merge = pd.DataFrame()

# df_customerとdf_receiptのデータを結合する
# sales_ymdをもとに年のデータを作成する
df_p_084_merge = pd.merge(df_customer, df_receipt, how='inner', on='customer_id')
df_p_084_merge['sales_year'] = df_p_084_merge['sales_ymd'].astype(str).str[:4].astype(int)

# 顧客データ（df_customer）の全顧客について、各年の売り上げを合計する
df_p_084_amount_year = df_p_084_merge.query('not customer_id.str.startswith("Z")', engine='python').groupby(['customer_id','sales_year']).agg({'amount':'sum'}).reset_index()

# 2019年のみの売上データを作成する
df_p_084_amount_2019 = df_p_084_amount_year[df_p_084_amount_year['sales_year'] == 2019]
df_p_084_amount_2019 = df_p_084_amount_2019.rename(columns={'amount':'sales_2019'})
df_p_084_amount_2019.drop(columns='sales_year',inplace=True)

# 全年合計の売上データを作成する
df_p_084_amount_all = df_p_084_amount_year.groupby(['customer_id'])['amount'].sum().reset_index()
df_p_084_amount_all = df_p_084_amount_all.rename(columns={'amount':'sales_all'})

# 2019年の売り上げと全年合計のデータをdf_customerに結合する
df_p_084 = pd.merge(df_customer, df_p_084_amount_2019, how='left', on='customer_id')
df_p_084 = pd.merge(df_p_084, df_p_084_amount_all, how='left', on='customer_id')

# 売上実績がない場合は0として扱う
df_p_084 = df_p_084.fillna({'sales_2019':0, 'sales_all':0})

# 割合を計算し、新たなデータを作成する
df_p_084['rate_sales_2019'] = df_p_084['sales_2019'] / df_p_084['sales_all']
df_p_084 = df_p_084.fillna({'rate_sales_2019':0})

# 計算した割合が0超のものを抽出する
# 結果を10件表示する
print(df_p_084[df_p_084['rate_sales_2019'] > 0].head(10))

# 作成したデータに欠損が存在しないことを確認する
print(df_p_084.isnull().any())

       customer_id customer_name  gender_cd gender birth_day  age postal_cd  \
2   CS031415000172       宇多田 貴美子          1     女性  19761004   42  151-0053   
6   CS015414000103         奥野 陽子          1     女性  19770809   41  136-0073   
12  CS011215000048         芦田 沙耶          1     女性  19920201   27  223-0062   
15  CS029415000023         梅田 里穂          1     女性  19760117   43  279-0043   
21  CS035415000029         寺沢 真希          9     不明  19770927   41  158-0096   
23  CS023513000066         神戸 そら          1     女性  19611217   57  210-0005   
24  CS035513000134         市川 美帆          1     女性  19600327   59  156-0053   
27  CS001515000263         高松 夏空          1     女性  19621109   56  144-0051   
30  CS006415000279          末永 綾          1     女性  19691003   49  224-0037   
32  CS031415000106        宇野 由美子          1     女性  19700226   49  151-0053   

                     address application_store_cd  application_date  \
2        東京都渋谷区代々木**********               S13031          

---
> ★P-085: 顧客データ（df_customer）の全顧客に対し、郵便番号（postal_cd）を用いてジオコードデータ（df_geocode）を紐付け、新たな顧客データを作成せよ。ただし、1つの郵便番号（postal_cd）に複数の経度（longitude）、緯度（latitude）情報が紐づく場合は、経度（longitude）、緯度（latitude）の平均値を算出して使用すること。また、作成結果を確認するために結果を10件表示せよ。

In [48]:
df_p_085 = pd.DataFrame()

# 郵便番号（postal_cd）でグループ化し、経度（longitude）緯度（latitude）の平均を算出する
p_58_df_geocode_avg = df_geocode.groupby('postal_cd').agg({'longitude':'mean', 'latitude':'mean'}).reset_index()
p_58_df_geocode_avg.rename(columns={'longitude':'m_longitude', 'latitude':'m_latitude'},inplace=True)

# 顧客データ（df_customer）の全顧客に対し、郵便番号（postal_cd）を用いてジオコードデータ（df_geocode）を紐付ける
df_p_085 = pd.merge(df_customer, df_geocode, how='inner', on='postal_cd')
df_p_085 = pd.merge(df_p_085, p_58_df_geocode_avg, how='inner', on='postal_cd')
df_p_085.drop(columns=['longitude', 'latitude'],inplace=True)

# 結果を10件表示する
print(df_p_085.head(10))

      customer_id customer_name  gender_cd gender birth_day  age postal_cd  \
0  CS021313000114        大野 あや子          1     女性  19810429   37  259-1113   
1  CS021303000023          堀 一徳          0     男性  19800405   38  259-1113   
2  CS021303000007         石倉 俊二          0     男性  19870704   31  259-1113   
3  CS021313000183         草野 未來          1     女性  19861221   32  259-1113   
4  CS021314000098        筒井 れいな          1     女性  19850921   33  259-1113   
5  CS021314000093         江藤 美佐          1     女性  19860603   32  259-1113   
6  CS021413000049         大野 幸子          1     女性  19730417   45  259-1113   
7  CS037613000071         六角 雅彦          9     不明  19520401   66  136-0076   
8  CS015415000209         大谷 倫子          1     女性  19701125   48  136-0076   
9  CS037614000045         長沢 麻緒          1     女性  19520619   66  136-0076   

              address_x application_store_cd  application_date  ...  \
0  神奈川県伊勢原市粟窪**********               S14021          20150905  ...   

---
> ★P-086: 085で作成した緯度経度つき顧客データに対し、会員申込店舗コード（application_store_cd）をキーに店舗データ（df_store）と結合せよ。そして申込み店舗の緯度（latitude）・経度情報（longitude)と顧客住所（address）の緯度・経度を用いて申込み店舗と顧客住所の距離（単位：km）を求め、顧客ID（customer_id）、顧客住所（address）、店舗住所（address）とともに表示せよ。計算式は以下の簡易式で良いものとするが、その他精度の高い方式を利用したライブラリを利用してもかまわない。結果は10件表示せよ。

In [49]:
#写経での回答
df_p_086 = pd.DataFrame()

# 085で作成した顧客データに対し、会員申込店舗コード（application_store_cd）をキーに店舗データ（df_store）と結合する
# 内部結合を用いてデータがないケースを除外する / df_store側は絡むめいをright_onで指定する
df_p_086 = pd.merge(df_p_085, df_store, how='inner', left_on='application_store_cd', right_on='store_cd')

# 申込み店舗の緯度（latitude）・経度情報（longitude)と顧客住所（address）の緯度・経度を用いて申込み店舗と顧客住所の距離（単位：km）を求める
# 計算式は下記の通り
def calc_distance(x1,y1,x2,y2):
    distance =6371 * math.acos(math.sin(math.radians(y1))* math.sin(math.radians(y2))+ math.cos(math.radians(y1)) * math.cos(math.radians(y2)) * math.cos(math.radians(x1) - math.radians(x2)))
    return distance

df_p_086['distance'] = df_p_086[['m_longitude','m_latitude','longitude','latitude']].apply(lambda x: calc_distance(x[0], x[1], x[2], x[3]), axis=1)

# 顧客ID（customer_id）、顧客住所（address）、店舗住所（address）とともに表示する
# 結果を10件表示する
print(df_p_086[['customer_id','address','address','distance']].head(10))

      customer_id         address         address  distance
0  CS021313000114  神奈川県伊勢原市伊勢原四丁目  神奈川県伊勢原市伊勢原四丁目  1.394409
1  CS021303000023  神奈川県伊勢原市伊勢原四丁目  神奈川県伊勢原市伊勢原四丁目  1.394409
2  CS021303000007  神奈川県伊勢原市伊勢原四丁目  神奈川県伊勢原市伊勢原四丁目  1.394409
3  CS021313000183  神奈川県伊勢原市伊勢原四丁目  神奈川県伊勢原市伊勢原四丁目  1.394409
4  CS021314000098  神奈川県伊勢原市伊勢原四丁目  神奈川県伊勢原市伊勢原四丁目  1.394409
5  CS021314000093  神奈川県伊勢原市伊勢原四丁目  神奈川県伊勢原市伊勢原四丁目  1.394409
6  CS021413000049  神奈川県伊勢原市伊勢原四丁目  神奈川県伊勢原市伊勢原四丁目  1.394409
7  CS021313000025  神奈川県伊勢原市伊勢原四丁目  神奈川県伊勢原市伊勢原四丁目  0.474282
8  CS021413000022  神奈川県伊勢原市伊勢原四丁目  神奈川県伊勢原市伊勢原四丁目  0.474282
9  CS021413000094  神奈川県伊勢原市伊勢原四丁目  神奈川県伊勢原市伊勢原四丁目  0.474282


---
> ★P-087: 顧客データ（df_customer）では、異なる店舗での申込みなどにより同一顧客が複数登録されている。名前（customer_name）と郵便番号（postal_cd）が同じ顧客は同一顧客とみなして1顧客1レコードとなるように名寄せした名寄顧客データを作成し、顧客データの件数、名寄顧客データの件数、重複数を算出せよ。ただし、同一顧客に対しては売上金額合計が最も高いものを残し、売上金額合計が同一もしくは売上実績がない顧客については顧客ID（customer_id）の番号が小さいものを残すこととする。

In [50]:
#写経での回答
df_p_087 = pd.DataFrame()

# 顧客ごとの売上データを作成する
# 顧客データ（df_customer）と売り上げデータを結合する
df_p_087_customer_sales = df_receipt.query('not customer_id.str.startswith("Z")', engine='python').groupby('customer_id').agg(sum_amount=('amount','sum')).reset_index()
df_p_08_merge = pd.merge(df_customer, df_p_087_customer_sales, how='left', on='customer_id').sort_values(['sum_amount','customer_id'],ascending=[False,True])
df_p_08_merge['sum_amount'] = df_p_08_merge['sum_amount'].fillna(0)

# 名前（customer_name）と郵便番号（postal_cd）が同じ顧客（重複）は、売上金額合計が最も高いものを残す
# 売上金額合計が同一もしくは売上実績がない顧客｜顧客ID（customer_id）の番号が小さいものを残す
df_p_087 = df_p_08_merge.drop_duplicates(subset=['customer_name','postal_cd'],keep='first')

# 顧客データの件数、名寄顧客データの件数、重複数を算出する
print('名寄顧客データ:{}件'.format(df_p_087.shape[0]))
print('顧客データ:{}件'.format(df_p_08_merge.shape[0]))
print('重複:{}件'.format(df_p_08_merge.shape[0]-df_p_087.shape[0]))

# 名寄せデータを表示する
print(df_p_087.head(10))

名寄顧客データ:21941件
顧客データ:21971件
重複:30件
          customer_id customer_name  gender_cd gender birth_day  age  \
16905  CS017415000097         福士 千夏          1     女性  19730403   45   
12692  CS015415000185        岩淵 はるみ          1     女性  19730919   45   
13550  CS031414000051        長澤 沙知絵          1     女性  19730425   45   
4808   CS028415000007         紺野 あい          1     女性  19690728   49   
14205  CS001605000009         安部 耕司          0     男性  19521022   66   
14760  CS010214000010         高嶋 芽以          1     女性  19910219   28   
15709  CS006515000023        竹村 はるみ          1     女性  19630627   55   
6353   CS016415000141         西谷 愛梨          1     女性  19740506   44   
21458  CS011414000106          紺野 窈          1     女性  19721111   46   
20336  CS038415000104        城戸 しほり          1     女性  19710221   48   

      postal_cd                    address application_store_cd  \
16905  166-0014        東京都杉並区松ノ木**********               S13017   
12692  135-0043         東京都江東区塩浜******

---
> ★P-088: 087で作成したデータを元に、顧客データに統合名寄IDを付与したデータを作成せよ。ただし、統合名寄IDは以下の仕様で付与するものとする。
>
> - 重複していない顧客：顧客ID（customer_id）を設定
> - 重複している顧客：前設問で抽出したレコードの顧客IDを設定
> 
> 顧客IDのユニーク件数と、統合名寄IDのユニーク件数の差も確認すること。

In [51]:
# 写経での回答
df_p_088 = pd.DataFrame()

# 087で作成したデータと顧客データ（df_customer）を結合する
# 顧客名と郵便番号で重複を確認
# 内部結合で重複を排除｜df_p_87でキーの'customer_name','postal_cd'を指定しつつ、追加列の'customer_id'を指定
df_p_088 = pd.merge(df_customer, df_p_087[['customer_name','postal_cd','customer_id']],how='inner', on =['customer_name', 'postal_cd'])
df_p_088.rename(columns={'customer_id_x':'customer_id','customer_id_y':'integration_id'}, inplace=True)

# 顧客IDのユニーク件数と統合名寄IDのユニーク件数の差を確認する
print('ID数の差', len(df_p_088['customer_id'].unique()) - len(df_p_088['integration_id'].unique()))
print(df_p_088.head(10))

ID数の差 30
      customer_id customer_name  gender_cd gender birth_day  age postal_cd  \
0  CS021313000114        大野 あや子          1     女性  19810429   37  259-1113   
1  CS037613000071         六角 雅彦          9     不明  19520401   66  136-0076   
2  CS031415000172       宇多田 貴美子          1     女性  19761004   42  151-0053   
3  CS028811000001        堀井 かおり          1     女性  19330327   86  245-0016   
4  CS001215000145         田崎 美紀          1     女性  19950329   24  144-0055   
5  CS020401000016         宮下 達士          0     男性  19740915   44  174-0065   
6  CS015414000103         奥野 陽子          1     女性  19770809   41  136-0073   
7  CS029403000008          釈 人志          0     男性  19730817   45  279-0003   
8  CS015804000004         松谷 米蔵          0     男性  19310502   87  136-0073   
9  CS033513000180          安斎 遥          1     女性  19620711   56  241-0823   

                  address application_store_cd  application_date  \
0    神奈川県伊勢原市粟窪**********               S14021          20150905

---
> ★P-089: 売上実績がある顧客を、予測モデル構築のため学習用データとテスト用データに分割したい。それぞれ8:2の割合でランダムにデータを分割せよ。

In [83]:
# 写経による回答

# 分割に必要なライブラリをインポートする
from sklearn.model_selection import train_test_split

# 購買データから売上実績がある顧客リストを作成する
df_p_089_customer = df_receipt.groupby('customer_id').agg({'amount':sum}).reset_index()
df_p_089_customer = df_p_089_customer.query('amount > 0')

# 整理した顧客リストと顧客データを結合する
df_p_089 = pd.merge(df_customer, df_p_089_customer['customer_id'], how='inner', on='customer_id')
print(df_p_089[['customer_id', 'customer_name']].head(10))

# 結合したデータを分割する
df_p_089_train, df_p_089_test = train_test_split(df_p_089, test_size=0.2, random_state=71)

# 分割結果を表示する
print('学習データ割合: ', len(df_p_089_train) / len(df_p_089))
print('テストデータ割合: ', len(df_p_089_test) / len(df_p_089))

      customer_id customer_name
0  CS031415000172       宇多田 貴美子
1  CS001215000145         田崎 美紀
2  CS015414000103         奥野 陽子
3  CS033513000180          安斎 遥
4  CS011215000048         芦田 沙耶
5  CS040412000191         川井 郁恵
6  CS029415000023         梅田 里穂
7  CS009315000023         皆川 文世
8  CS035415000029         寺沢 真希
9  CS015315000033        福士 璃奈子
学習データ割合:  0.7999036840837949
テストデータ割合:  0.20009631591620516


---
> ★P-090: レシート明細データ（df_receipt）は2017年1月1日〜2019年10月31日までのデータを有している。売上金額（amount）を月次で集計し、学習用に12ヶ月、テスト用に6ヶ月の時系列モデル構築用データを3セット作成せよ。

In [84]:
# df_receiptから売上金額と年月日のみを抽出する
df_p_090_amount = df_receipt[['sales_ymd', 'amount']].copy()
df_p_090_amount['sales_ym'] = df_p_090_amount['sales_ymd'].astype('str').str[0:6] # sales_ymdからyyyymmのみを抽出する

# sales_ymで売上金額を合計する
df_p_090_amount = df_p_090_amount.groupby('sales_ym').agg({'amount':'sum'}).reset_index()


# 長期間データに対する多数のデータセットもループなどで処理できるように関数化
def split_data(df, train_size, test_size, slide_window, start_point):
    train_start = start_point * slide_window
    test_start = train_start + train_size
    return df[train_start:test_start], df[test_start:test_start + test_size]

df_p_090_train_1, df_p_090_test_1 = split_data(df_p_090_amount, train_size=12, test_size=6, slide_window=6, start_point=0)
df_p_090_train_2, df_p_090_test_2 = split_data(df_p_090_amount, train_size=12, test_size=6, slide_window=6, start_point=1)
df_p_090_train_3, df_p_090_test_3 = split_data(df_p_090_amount, train_size=12, test_size=6, slide_window=6, start_point=2)

---
> ★P-091: 顧客データ（df_customer）の各顧客に対し、売上実績がある顧客数と売上実績がない顧客数が1:1となるようにアンダーサンプリングで抽出せよ。

In [86]:
# 写経での回答
# アンダーサンプリング：多数派のデータを少数派のデータ数に合わせて削除する手法

from imblearn.under_sampling import RandomUnderSampler

# 売上実績がある顧客リストを作成する
df_p_091 = df_receipt.groupby('customer_id').agg({'amount':'sum'}).reset_index()
df_p_091 = pd.merge(df_customer, df_p_091, how='left', on='customer_id') #顧客データと顧客リストを結合する｜外部結合にし、売上実績がない顧客を残しておく
df_p_091['is_buy_flag'] = np.where(df_p_091['amount'].isnull(), 0, 1) #売上実績がない場合に0を、ある場合には1を'is_buy_flag'に投入する

# RandomUnderSamplerクラスのインスタンスを生成し、fit_sample(X, y)のようにすることで、yの値が均等になるようにアンダーサンプリングがXに対して行われます。
df_p_091_rs = RandomUnderSampler(random_state=71) #random_stateを設定することで再現性や乱数生成の一貫性を担保する
df_p_091_down_sampling, _ = df_p_091_rs.fit_resample(df_p_091, df_p_091.is_buy_flag)

print('0の件数', len(df_p_091_down_sampling.query('is_buy_flag == 0')))
print('1の件数', len(df_p_091_down_sampling.query('is_buy_flag == 1')))

0の件数 8306
1の件数 8306


---
> ★P-092: 顧客データ（df_customer）の性別について、第三正規形へと正規化せよ。

In [87]:
# 写経での回答
# 第三正規形：段階的な従属関係にする（性別と性別cd、その他のデータがdf_customer内に格納されているので切り出す）

df_p_092_gender = df_customer[['gender_cd', 'gender']].drop_duplicates()
df_p_092_customer = df_customer.drop(columns='gender')

print(df_p_092_gender.head(10))
print(df_p_092_customer.head(10))

   gender_cd gender
0          1     女性
1          9     不明
5          0     男性
      customer_id customer_name  gender_cd   birth_day  age postal_cd  \
0  CS021313000114        大野 あや子          1  1981-04-29   37  259-1113   
1  CS037613000071         六角 雅彦          9  1952-04-01   66  136-0076   
2  CS031415000172       宇多田 貴美子          1  1976-10-04   42  151-0053   
3  CS028811000001        堀井 かおり          1  1933-03-27   86  245-0016   
4  CS001215000145         田崎 美紀          1  1995-03-29   24  144-0055   
5  CS020401000016         宮下 達士          0  1974-09-15   44  174-0065   
6  CS015414000103         奥野 陽子          1  1977-08-09   41  136-0073   
7  CS029403000008          釈 人志          0  1973-08-17   45  279-0003   
8  CS015804000004         松谷 米蔵          0  1931-05-02   87  136-0073   
9  CS033513000180          安斎 遥          1  1962-07-11   56  241-0823   

                  address application_store_cd  application_date     status_cd  
0    神奈川県伊勢原市粟窪**********          

---
> ★P-093: 商品データ（df_product）では各カテゴリのコード値だけを保有し、カテゴリ名は保有していない。カテゴリデータ（df_category）と組み合わせて非正規化し、カテゴリ名を保有した新たな商品データを作成せよ。

In [91]:
df_p_093 = pd.merge(df_product, df_category[['category_small_cd','category_major_name','category_medium_name','category_small_name']], how='inner', on='category_small_cd')

print(df_p_093.head(3))

   product_cd  category_major_cd  category_medium_cd  category_small_cd  \
0  P040101001                  4                 401              40101   
1  P040101002                  4                 401              40101   
2  P040101003                  4                 401              40101   

   unit_price  unit_cost category_major_name category_medium_name  \
0       198.0      149.0                  惣菜                  御飯類   
1       218.0      164.0                  惣菜                  御飯類   
2       230.0      173.0                  惣菜                  御飯類   

  category_small_name  
0                 弁当類  
1                 弁当類  
2                 弁当類  


---
> ★P-096: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|無し|UTF-8|
> 
> ファイル出力先のパスは以下のようにすること。
> 
> |出力先|
> |:--:|
> |./data|

In [94]:
df_p_093.to_csv('../df_p_093.csv', encoding='utf-8',header=False, index=False)


---
> ★P-098: 096で作成した以下形式のファイルを読み込み、データを3件を表示させて正しく取り込まれていることを確認せよ。
> 
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|ヘッダ無し|UTF-8|

In [96]:
#写経にて回答

# read_csvでcsvを読み込み、下記の引数で指定
# dtypeでカラムのデータ型を指定する
# headerがないので、Noneを指定
df_p_098 = pd.read_csv('../df_p_093.csv',
                       dtype={1:str,
                              2:str,
                              3:str},
                       encoding='utf-8',header=None)

# カラムがないため、直接指定する
df_p_098.columns = ['product_cd','category_major_cd',
                           'category_medium_cd', 'category_small_cd',
                           'unit_price','unit_cost','category_major_name',
                           'category_medium_name', 'category_small_name']

print(df_p_098.head(10))


   product_cd category_major_cd category_medium_cd category_small_cd  \
0  P040101001                 4                401             40101   
1  P040101002                 4                401             40101   
2  P040101003                 4                401             40101   
3  P040101004                 4                401             40101   
4  P040101005                 4                401             40101   
5  P040101006                 4                401             40101   
6  P040101007                 4                401             40101   
7  P040101008                 4                401             40101   
8  P040101009                 4                401             40101   
9  P040101010                 4                401             40101   

   unit_price  unit_cost category_major_name category_medium_name  \
0       198.0      149.0                  惣菜                  御飯類   
1       218.0      164.0                  惣菜                  御飯類   


---
> P-001: レシート明細データ（df_receipt）から全項目の先頭10件を表示し、どのようなデータを保有しているか目視で確認せよ。

---
> P-002: レシート明細データ（df_receipt）から売上年月日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示せよ。

---
> P-004: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"

---
> P-005: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上

---
> P-006: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上数量（quantity）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上または売上数量（quantity）が5以上

---
> P-008: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 商品コード（product_cd）が"P071401019"以外

---
> P-009: 以下の処理において、出力結果を変えずにORをANDに書き換えよ。
> 
> `df_store.query('not(prefecture_cd == "13" | floor_area > 900)')`

---
> P-010: 店舗データ（df_store）から、店舗コード（store_cd）が"S14"で始まるものだけ全項目抽出し、10件表示せよ。

---
> P-011: 顧客データ（df_customer）から顧客ID（customer_id）の末尾が1のものだけ全項目抽出し、10件表示せよ。

---
> P-012: 店舗データ（df_store）から、住所 (address) に"横浜市"が含まれるものだけ全項目表示せよ。

---
> P-014: 顧客データ（df_customer）から、ステータスコード（status_cd）の末尾が数字の1〜9で終わるデータを全項目抽出し、10件表示せよ。

---
> P-015: 顧客データ（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まり、末尾が数字の1〜9で終わるデータを全項目抽出し、10件表示せよ。

---
> P-017: 顧客データ（df_customer）を生年月日（birth_day）で高齢順にソートし、先頭から全項目を10件表示せよ。

---
> P-018: 顧客データ（df_customer）を生年月日（birth_day）で若い順にソートし、先頭から全項目を10件表示せよ。

---
> P-019: レシート明細データ（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭から10件表示せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合は同一順位を付与するものとする。

---
> P-020: レシート明細データ（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭から10件表示せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合でも別順位を付与すること。

---
> P-021: レシート明細データ（df_receipt）に対し、件数をカウントせよ。

---
> P-023: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）と売上数量（quantity）を合計せよ。

---
> P-024: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上年月日（sales_ymd）を求め、10件表示せよ。

---
> P-025: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も古い売上年月日（sales_ymd）を求め、10件表示せよ。

---
> P-028: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の中央値を計算し、降順でTOP5を表示せよ。

---
> P-029: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに商品コード（product_cd）の最頻値を求め、10件表示させよ。

---
> P-030: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の分散を計算し、降順で5件表示せよ。

---
> P-031: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標準偏差を計算し、降順で5件表示せよ。

TIPS:

PandasとNumpyでddofのデフォルト値が異なることに注意しましょう
```
Pandas：
DataFrame.std(self, axis=None, skipna=None, level=None, ddof=1, numeric_only=None, **kwargs)
Numpy:
numpy.std(a, axis=None, dtype=None, out=None, ddof=0, keepdims=)
```

---
> P-032: レシート明細データ（df_receipt）の売上金額（amount）について、25％刻みでパーセンタイル値を求めよ。

---
> P-033: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、330以上のものを抽出せよ。

---
> P-034: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求めよ。ただし、顧客IDが"Z"から始まるものは非会員を表すため、除外して計算すること。

---
> P-035: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求め、平均以上に買い物をしている顧客を抽出し、10件表示せよ。ただし、顧客IDが"Z"から始まるものは非会員を表すため、除外して計算すること。

---
> P-037: 商品データ（df_product）とカテゴリデータ（df_category）を内部結合し、商品データの全項目とカテゴリデータのカテゴリ小区分名（category_small_name）を10件表示せよ。

---
> P-038: 顧客データ（df_customer）とレシート明細データ（df_receipt）から、顧客ごとの売上金額合計を求め、10件表示せよ。ただし、売上実績がない顧客については売上金額を0として表示させること。また、顧客は性別コード（gender_cd）が女性（1）であるものを対象とし、非会員（顧客IDが"Z"から始まるもの）は除外すること。

---
> P-039: レシート明細データ（df_receipt）から、売上日数の多い顧客の上位20件を抽出したデータと、売上金額合計の多い顧客の上位20件を抽出したデータをそれぞれ作成し、さらにその2つを完全外部結合せよ。ただし、非会員（顧客IDが"Z"から始まるもの）は除外すること。

---
> P-040: 全ての店舗と全ての商品を組み合わせたデータを作成したい。店舗データ（df_store）と商品データ（df_product）を直積し、件数を計算せよ。

---
> P-041: レシート明細データ（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、前回売上があった日からの売上金額増減を計算せよ。そして結果を10件表示せよ。

---
> P-042: レシート明細データ（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、各日付のデータに対し、前回、前々回、3回前に売上があった日のデータを結合せよ。そして結果を10件表示せよ。

---
> P-043： レシート明細データ（df_receipt）と顧客データ（df_customer）を結合し、性別コード（gender_cd）と年代（ageから計算）ごとに売上金額（amount）を合計した売上サマリデータを作成せよ。性別コードは0が男性、1が女性、9が不明を表すものとする。
>
> ただし、項目構成は年代、女性の売上金額、男性の売上金額、性別不明の売上金額の4項目とすること（縦に年代、横に性別のクロス集計）。また、年代は10歳ごとの階級とすること。

---
> P-044： 043で作成した売上サマリデータ（df_sales_summary）は性別の売上を横持ちさせたものであった。このデータから性別を縦持ちさせ、年代、性別コード、売上金額の3項目に変換せよ。ただし、性別コードは男性を"00"、女性を"01"、不明を"99"とする。

---
> P-046: 顧客データ（df_customer）の申し込み日（application_date）はYYYYMMDD形式の文字列型でデータを保有している。これを日付型に変換し、顧客ID（customer_id）とともに10件表示せよ。

---
> P-047: レシート明細データ（df_receipt）の売上日（sales_ymd）はYYYYMMDD形式の数値型でデータを保有している。これを日付型に変換し、レシート番号（receipt_no）、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

---
> P-048: レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）は数値型のUNIX秒でデータを保有している。これを日付型に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

---
> P-049: レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）を日付型に変換し、「年」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

---
> P-050: レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）を日付型に変換し、「月」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。なお、「月」は0埋め2桁で取り出すこと。

---
> P-060: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を最小値0、最大値1に正規化して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

---
> P-061: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を常用対数化（底10）して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

---
> P-062: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を自然対数化（底e）して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

---
> P-064: 商品データ（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益率の全体平均を算出せよ。ただし、単価と原価には欠損が生じていることに注意せよ。

---
> P-066: 商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を丸めること（四捨五入または偶数への丸めで良い）。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

---
> P-067: 商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を切り上げること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

---
> P-068: 商品データ（df_product）の各商品について、消費税率10％の税込み金額を求めよ。1円未満の端数は切り捨てとし、結果を10件表示せよ。ただし、単価（unit_price）には欠損が生じていることに注意せよ。

---
> P-071: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過月数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1ヶ月未満は切り捨てること。

---
> P-072: レシート明細データ（df_receipt）の売上日（df_customer）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過年数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1年未満は切り捨てること。

---
> P-073: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からのエポック秒による経過時間を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。なお、時間情報は保有していないため各日付は0時0分0秒を表すものとする。

---
> P-074: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、当該週の月曜日からの経過日数を計算し、売上日、直前の月曜日付とともに10件表示せよ（sales_ymdは数値でデータを保持している点に注意）。

---
> P-079: 商品データ（df_product）の各項目に対し、欠損数を確認せよ。

---
> P-080: 商品データ（df_product）のいずれかの項目に欠損が発生しているレコードを全て削除した新たな商品データを作成せよ。なお、削除前後の件数を表示させ、079で確認した件数だけ減少していることも確認すること。

---
> P-082: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの中央値で補完した新たな商品データを作成せよ。なお、中央値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

---
> P-094: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|有り|UTF-8|
> 
> ファイル出力先のパスは以下のようにすること
> 
> |出力先|
> |:--:|
> |./data|

---
> P-095: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|有り|CP932|
> 
> ファイル出力先のパスは以下のようにすること。
> 
> |出力先|
> |:--:|
> |./data|

---
> P-097: 094で作成した以下形式のファイルを読み込み、データを3件を表示させて正しく取り込まれていることを確認せよ。
> 
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|有り|UTF-8|

---
> P-099: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |TSV（タブ区切り）|有り|UTF-8|
> 
> ファイル出力先のパスは以下のようにすること
> 
> |出力先|
> |:--:|
> |./data|

---
> P-100: 099で作成した以下形式のファイルを読み込み、データを3件を表示させて正しく取り込まれていることを確認せよ。
> 
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |TSV（タブ区切り）|有り|UTF-8|

# これで１００本終わりです。おつかれさまでした！